In [1]:
import loader as l
import plots as pl
import analysis as a
import patterns as pt
import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
tickers = 'COP'

In [3]:
stock_data = l.load_data(tickers)
stock_data = l.check_clean_data(stock_data, verbose=True)

From COP dataset were removed 5 rows with missing or incorrect data


In [4]:
stock_data[tickers]['splits']

Date
2005-06-02 00:00:00-04:00    2.000000
2012-05-01 00:00:00-04:00    1.311791
Name: Stock Splits, dtype: float64

In [5]:
df = stock_data[tickers]['historical_data']
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1995-01-03 00:00:00-05:00,4.896,4.952,4.878,4.878,965478,0.0,0.0
1995-01-04 00:00:00-05:00,4.896,4.896,4.803,4.859,1478388,0.0,0.0
1995-01-05 00:00:00-05:00,4.859,4.859,4.803,4.822,1209471,0.0,0.0
1995-01-06 00:00:00-05:00,4.822,4.840,4.784,4.803,1535583,0.0,0.0
1995-01-09 00:00:00-05:00,4.766,4.766,4.635,4.653,1293164,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-28 00:00:00-04:00,115.400,115.570,113.640,114.380,7191800,0.0,0.0
2024-07-01 00:00:00-04:00,115.040,115.460,113.650,114.360,3925100,0.0,0.0
2024-07-02 00:00:00-04:00,115.380,115.610,113.330,114.120,3836700,0.0,0.0


In [6]:
pl.plot_chart(df, ticker=tickers)

In [7]:
support_df = a.calculate_ATR(df)

/Users/carlonestor/Documents/GitHub/statistical_analysis_candlestick_patterns/analysis.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
pattern_name = "Bearish Harami"  # replace with the desired pattern name

# check if the pattern exists in the patterns dictionary
if pattern_name in pt.patterns:
    pattern_function = pt.patterns[pattern_name]['function']
    # use the function to obtain the mask of the pattern dates
    mask = pattern_function(pd.concat([df, support_df], axis=1))
    # filter the mask to ensure consistency in the future analysis
    mask = pt.filter_mask(mask, step=10)
    print(mask.sum())
else:
    print(f"The pattern '{pattern_name}' does not exist.")

89


In [9]:
pl.plot_close_with_patterns(df, tickers, mask)

In [10]:
pl.plot_patterns(df, mask, num_candles=pt.patterns[pattern_name]['candles'], ticker=tickers, pattern_name=pattern_name)

In [11]:
returns = a.calculate_cumReturns_periods(df, mask)
log_returns = a.calculate_log_cumReturns_periods(df, mask)
# round the returns to 3 decimal places and multiply by 100 to make percentages more readable
rounded_returns = {k: [round(100*r,3) for r in v] for k, v in returns.items()}
print(returns.keys())
print(len(returns[1]))
print(returns[1])

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
89
[0.00693533270852857, -0.023326572008113455, -0.026768968456948, -0.01581410359851539, 0.006227047740699298, -0.0059426533947407574, 0.015660809778456866, -0.006294637453092798, -0.006270348486675462, -0.008631750845678274, -0.004040667361835205, 0.006569976447254128, -0.007309336643221615, 0.022315411112775203, 0.0040490797546011625, 0.00881977486364149, -0.03363441882576652, -0.005746536685479648, 0.007626310772163972, -0.013333333333333365, 0.002695417789757355, -0.01571164510166358, 0.0018867924528301486, 0.011688582759340439, -0.014778823767587714, 0.004897159647404401, -0.011661807580174852, -0.004604758250191767, -0.01837985023825729, -0.02582311168495806, 0.001973684210526391, -0.0036900369003691432, -0.001144819690898659, 0.007467994310099477, -0.007729861676159387, 0.02039966694421324, 0.020497803806735077, -0.005907172995780415, -0.016284846812034126, -0.08575140753572977, 0.0018921475875117856, 0.0017574692442882914, 0.001702610

In [12]:
pl.plot_return_distributions(rounded_returns)

In [13]:
avg_returns = a.calculate_average_return(rounded_returns)
median_returns = a.calculate_median_return(rounded_returns)
win_rate = a.calculate_win_rate(rounded_returns)

In [14]:
pl.plot_original_stats(avg_returns, median_returns, win_rate)

In [15]:
from scipy.stats import ks_2samp, anderson_ksamp

num_samples = len(rounded_returns[1]) if len(rounded_returns[1]) > 100 else 1000
num_samples = 500

bootstrap_returns = a.bootstrap_sample(rounded_returns, dim_sample=num_samples)
kde_returns = a.kde_sample(rounded_returns, dim_sample=num_samples)
bayesian_returns = a.bayesian_sample(rounded_returns, dim_sample=num_samples)
mixed_returns = a.mixed_sample(rounded_returns, dim_sample=num_samples)

Processing day 1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 2


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 10


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


In [16]:
results = []

for day in range(1,11):
    original_day = rounded_returns[day]
    bootstrap_day = bootstrap_returns[day]
    kde_day = kde_returns[day]
    bayesian_day = bayesian_returns[day]
    mixed_day = mixed_returns[day]

    # K-S Test
    ks_bootstrap = ks_2samp(original_day, bootstrap_day).pvalue
    ks_kde = ks_2samp(original_day, kde_day).pvalue
    ks_bayesian = ks_2samp(original_day, bayesian_day).pvalue
    ks_mixed = ks_2samp(original_day, mixed_day).pvalue
    # Anderson-Darling Test
    ad_bootstrap = anderson_ksamp([original_day, bootstrap_day]).significance_level
    ad_kde = anderson_ksamp([original_day, kde_day]).significance_level
    ad_bayesian = anderson_ksamp([original_day, bayesian_day]).significance_level
    ad_mixed = anderson_ksamp([original_day, mixed_day]).significance_level

    results.append({
        "day": day,
        "ks_bootstrap": ks_bootstrap,
        "ks_kde": ks_kde,
        "ks_bayesian": ks_bayesian,
        "ks_mixed": ks_mixed,
        "ad_bootstrap": ad_bootstrap,
        "ad_kde": ad_kde,
        "ad_bayesian": ad_bayesian,
        "ad_mixed": ad_mixed
    })

print(results)

[{'day': 1, 'ks_bootstrap': 0.9999999999954848, 'ks_kde': 0.814570807042934, 'ks_bayesian': 0.8335852413775582, 'ks_mixed': 0.9644644307718878, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 2, 'ks_bootstrap': 1.0, 'ks_kde': 0.7689328256247117, 'ks_bayesian': 0.8865680383187582, 'ks_mixed': 0.9999195982445236, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 3, 'ks_bootstrap': 0.9999980642550984, 'ks_kde': 0.8966635772102304, 'ks_bayesian': 0.8870996074577552, 'ks_mixed': 0.998167007346763, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 4, 'ks_bootstrap': 0.9859809416859856, 'ks_kde': 0.879894126126497, 'ks_bayesian': 0.8142319441116003, 'ks_mixed': 0.9671259819461394, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 5, 'ks_bootstrap': 0.9999999999972388, 'ks_kde': 0.9333054038437643, 'ks_bayesian': 0.9533207263134669, 'ks_mixed': 0.994

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_8741/297188851.py:16: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_8741/297188851.py:17: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_8741/297188851.py:18: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_8741/297188851.py:19: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)



In [17]:
pl.qq_plot(rounded_returns, kde_returns)

In [18]:
pl.qq_plot(rounded_returns, bayesian_returns)

In [19]:
pl.qq_plot(rounded_returns, bootstrap_returns)

In [20]:
pl.qq_plot(rounded_returns, mixed_returns)

In [21]:
pl.plot_return_distributions(kde_returns)

In [22]:
pl.plot_return_distributions(bayesian_returns)

In [23]:
pl.plot_return_distributions(mixed_returns)

In [15]:
baseline_returns = a.generate_random_returns(df, dim_sample=500, n_iterations=1000)

Starting calculating returns...
Generated 1 samples.
Generated 2 samples.
Generated 3 samples.
Generated 4 samples.
Generated 5 samples.
Generated 6 samples.
Generated 7 samples.
Generated 8 samples.
Generated 9 samples.
Generated 10 samples.
Generated 11 samples.
Generated 12 samples.
Generated 13 samples.
Generated 14 samples.
Generated 15 samples.
Generated 16 samples.
Generated 17 samples.
Generated 18 samples.
Generated 19 samples.
Generated 20 samples.
Generated 21 samples.
Generated 22 samples.
Generated 23 samples.
Generated 24 samples.
Generated 25 samples.
Generated 26 samples.
Generated 27 samples.
Generated 28 samples.
Generated 29 samples.
Generated 30 samples.
Generated 31 samples.
Generated 32 samples.
Generated 33 samples.
Generated 34 samples.
Generated 35 samples.
Generated 36 samples.
Generated 37 samples.
Generated 38 samples.
Generated 39 samples.
Generated 40 samples.
Generated 41 samples.
Generated 42 samples.
Generated 43 samples.
Generated 44 samples.
Generated

In [16]:
baseline_results = a.calculate_metrics(baseline_returns)

In [17]:
bayesian_pool_returns = a.generate_bayesian_pool(rounded_returns, pool_size=150000)

Processing day 1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 2


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 10


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


In [18]:
pattern_returns = a.generate_pattern_returns(rounded_returns, dim_sample=500, n_iterations=1000, bayesian_pool=bayesian_pool_returns)

In [23]:
pattern_results = a.calculate_metrics(pattern_returns)

In [26]:
pl.plot_metric_distributions(baseline_results, metric_name='average_return')

In [27]:
pl.plot_metric_distributions(pattern_results, metric_name='average_return')

In [32]:
pl.compare_boxplots(baseline_results, pattern_results, metric='average_return')

In [34]:
pl.compare_boxplots(baseline_results, pattern_results, metric='median_return')